In [1]:
from datetime import datetime, timedelta, timezone
import ccxt
import plotly.express as px
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO, format='%(asctime)s|%(levelname)s|%(message)s')
log = logging.getLogger(__name__)




In [2]:
def dt2ts(dt):
    return int(dt.timestamp() * 1000)

def ts2dt(ts):
    return datetime.fromtimestamp(ts / 1000)
def str2dt(s):
    return datetime.fromisoformat(s)
def dt2str(dt):
    return dt.isoformat()

In [3]:
from pathlib import Path
import json

for exchange in ccxt.exchanges:
    try:
        print(exchange)
        markets = getattr(ccxt, exchange)().load_markets()
        Path(f"exchanges/{exchange}.json").write_text(json.dumps(markets, indent=4))
    except Exception as e:
        print(f"{exchange} failed with {e}")
        continue

ace
alpaca
alpaca failed with alpaca {"message": "forbidden."}
ascendex


KeyboardInterrupt: 

In [3]:
from contextlib import contextmanager


@contextmanager
def supress():
    try:
        yield
    except Exception as e:
        log.exception(f"{exchange} failed with {e}")
        return

In [4]:
def print_futures(api, binance_api, results):
    for key, info in api.load_markets().items():
        with supress():
            if not info["future"] or not info["inverse"] or not info["active"]:
                continue
            if api.id == "deribit" and info["info"]["kind"] != "future":
                continue
                
            Path(f"infos/{api.id}_{info['id']}.json").write_text(json.dumps(info, indent=4))
            future_id = info["id"]
            future_price = api.fetch_ticker(future_id)["last"]
            if future_price is None:
                print(f"{future_id=} Last price is none")
                continue
            
            base_symbol = info["settle"] + "USDT"
            base_price = binance_api.fetch_ticker(base_symbol)["last"]
            period_yield = (future_price - base_price) / base_price * 100
            expiry_date = datetime.fromisoformat(info["expiryDatetime"][:len("2024-06-28")])
            days_till_expiry = (expiry_date - datetime.now()).days
            year_yield = period_yield / days_till_expiry * 365 if days_till_expiry else float("nan")
            result = dict(exchange_id=api.id,
                id=key,
                future_id=future_id,
                future_price=future_price,
                base_symbol=base_symbol,
                base_price=base_price,
                period_yield=period_yield,
                days_till_expiry=days_till_expiry,
                expiry_date=expiry_date,
                year_yield=year_yield
            )
            results.append(result)
            print(result)
        

In [5]:
# alpaca" # ccxt.base.errors.PermissionDenied: alpaca {"message": "forbidden."}
# coinbase # ccxt.base.errors.AuthenticationError: coinbase requires "apiKey" credential
# mercado # GET https://www.mercadobitcoin.net/api/coins 403 Forbidden
# zaif # GET https://api.zaif.jp/api/1/currency_pairs/all 403 Forbidden 
# binancecoinm # same as binance
EXCHANGE_IDS = "binance bitget bitmex bybit deribit htx krakenfutures kucoinfutures okx".split()

In [33]:
from pathlib import Path
import json
binance_api = ccxt.binance()
results = []
for exchange_id in EXCHANGE_IDS:
    with supress():
        api = getattr(ccxt, exchange_id)()
        print_futures(api=api, binance_api=binance_api, results=results)


{'exchange_id': 'binance', 'id': 'BTC/USD:BTC-240927', 'future_id': 'BTCUSD_240927', 'future_price': 62435.0, 'base_symbol': 'BTCUSDT', 'base_price': 60828.11, 'period_yield': 2.641689837149304, 'days_till_expiry': 90, 'expiry_date': datetime.datetime(2024, 9, 27, 0, 0), 'year_yield': 10.713519895105511}
{'exchange_id': 'binance', 'id': 'BTC/USD:BTC-241227', 'future_id': 'BTCUSD_241227', 'future_price': 64563.3, 'base_symbol': 'BTCUSDT', 'base_price': 60828.1, 'period_yield': 6.140583052898257, 'days_till_expiry': 181, 'expiry_date': datetime.datetime(2024, 12, 27, 0, 0), 'year_yield': 12.38294372545781}
{'exchange_id': 'binance', 'id': 'ETH/USD:ETH-240927', 'future_id': 'ETHUSD_240927', 'future_price': 3507.4, 'base_symbol': 'ETHUSDT', 'base_price': 3406.02, 'period_yield': 2.9764945596326537, 'days_till_expiry': 90, 'expiry_date': datetime.datetime(2024, 9, 27, 0, 0), 'year_yield': 12.071339047399094}
{'exchange_id': 'binance', 'id': 'ETH/USD:ETH-241227', 'future_id': 'ETHUSD_241227'

In [34]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame(results).sort_values(["days_till_expiry", "period_yield"]))

,exchange_id,id,future_id,future_price,base_symbol,base_price,period_yield,days_till_expiry,expiry_date,year_yield
27,bitmex,BTC/USD:BTC-240628,XBTM24,61616.17000,BTCUSDT,60822.33000,1.305179,-1,2024-06-28,-476.390168
48,htx,BTC/USD:BTC-240705,BTC240705,60767.46000,BTCUSDT,60870.00000,-0.168457,6,2024-07-05,-10.247823
51,htx,ETH/USD:ETH-240705,ETH240705,3402.94700,ETHUSDT,3407.95000,-0.146804,6,2024-07-05,-8.930564
54,htx,TRX/USD:TRX-240705,TRX240705,0.12312,TRXUSDT,0.12318,-0.048709,6,2024-07-05,-2.963143
63,okx,ETH/USD:ETH-240705,ETH-USD-240705,3407.66000,ETHUSDT,3408.49000,-0.024351,6,2024-07-05,-1.481350
32,deribit,ETH/USD:ETH-240705,ETH-5JUL24,3406.75000,ETHUSDT,3406.32000,0.012624,6,2024-07-05,0.767935
57,okx,BTC/USD:BTC-240705,BTC-USD-240705,60903.10000,BTCUSDT,60872.00000,0.051091,6,2024-07-05,3.108024
40,deribit,BTC/USD:BTC-240705,BTC-5JUL24,60885.00000,BTCUSDT,60848.00000,0.060807,6,2024-07-05,3.699108
52,htx,ETH/USD:ETH-240712,ETH240712,3408.06600,ETHUSDT,3407.79000,0.008099,13,2024-07-12,0.227398
55,htx,TRX/USD:TRX-240712,TRX240712,0.12326,TRXUSDT,0.12318,0.064946,13,2024-07-12,1.823473


In [19]:

def to_df(data):
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='ms', utc=True)
    df.set_index('date', inplace=True)
    df = df.sort_index(ascending=True)
    return df

In [20]:

def get_prices(exchange_id: str, symbol: str, since: datetime, until: datetime, timeframe: str ='1h') -> pd.DataFrame:
    exchange = getattr(ccxt, exchange_id)()
    all_candles = []

    # Handle pagination to fetch all historical data within the given timeframe
    since_ts = dt2ts(since)
    until_ts = dt2ts(until)
    
    while since_ts < until_ts:
        candles = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, since=since_ts, limit=100)
        if not candles:
            break
        print(f"received {len(candles)} candles {ts2dt(candles[0][0])}->{ts2dt(candles[-1][0])}")
        since_ts = candles[-1][0] + 1  # Prepare 'since' for the next iteration
        all_candles += candles
        
    return all_candles


In [21]:
def merge(btc_fut_df: pd.DataFrame, btc_df: pd.DataFrame, expire: datetime) -> pd.DataFrame:
    merged_df = pd.merge(btc_df["close"], btc_fut_df["close"], left_index=True, right_index=True)
    merged_df.columns = ["spot", "futures"]
    merged_df["days_to_expiry"] = (expire - merged_df.index).days
    merged_df["yield"] = round(merged_df.futures / merged_df.spot * 100 - 100, 2)
    merged_df["apy"] = round((merged_df.futures / merged_df.spot * 100 - 100) / merged_df.days_to_expiry * 365, 2)
    merged_df["apy_quarter"] = round((merged_df.futures / merged_df.spot * 100 - 100) * 4, 2)
    return merged_df

In [31]:
since = str2dt('2024-06-20 00:00:00+00:00')
expire = str2dt('2024-06-24 00:00:00+00:00')
until = datetime.now(tz=timezone.utc)
timeframe = '1m'

In [32]:
btc_fut_df = to_df(get_prices(exchange_id="okx", symbol='ETC-USD-240927', since=since, until=until, timeframe=timeframe))
btc_df = to_df(get_prices(exchange_id="binance", symbol='ETCUSDT', since=since, until=until, timeframe=timeframe))
merged_df = merge(btc_fut_df=btc_fut_df, btc_df=btc_df, expire=expire)
print(f"{len(btc_fut_df)=} {len(btc_df)=} {len(merged_df)=}")
display(merged_df)
px.line(merged_df, y=["yield"], title="yield")


received 100 candles 2024-06-20 03:00:00->2024-06-20 04:39:00
received 100 candles 2024-06-20 04:40:00->2024-06-20 06:19:00
received 100 candles 2024-06-20 06:20:00->2024-06-20 07:59:00
received 100 candles 2024-06-20 08:00:00->2024-06-20 09:39:00
received 100 candles 2024-06-20 09:40:00->2024-06-20 11:19:00
received 100 candles 2024-06-20 11:20:00->2024-06-20 12:59:00
received 100 candles 2024-06-20 13:00:00->2024-06-20 14:39:00
received 100 candles 2024-06-20 14:40:00->2024-06-20 16:19:00
received 100 candles 2024-06-20 16:20:00->2024-06-20 17:59:00
received 100 candles 2024-06-20 18:00:00->2024-06-20 19:39:00
received 100 candles 2024-06-20 19:40:00->2024-06-20 21:19:00
received 100 candles 2024-06-20 21:20:00->2024-06-20 22:59:00
received 100 candles 2024-06-20 23:00:00->2024-06-21 00:39:00
received 100 candles 2024-06-21 00:40:00->2024-06-21 02:19:00
received 100 candles 2024-06-21 02:20:00->2024-06-21 03:59:00
received 100 candles 2024-06-21 04:00:00->2024-06-21 05:39:00
received

,spot,futures,days_to_expiry,yield,apy,apy_quarter
date,,,,,,
2024-06-20 00:00:00+00:00,23.96,24.47,4,2.13,194.23,8.51
2024-06-20 00:01:00+00:00,23.91,24.47,3,2.34,284.96,9.37
2024-06-20 00:02:00+00:00,23.90,24.47,3,2.38,290.17,9.54
2024-06-20 00:03:00+00:00,23.92,24.47,3,2.30,279.75,9.20
2024-06-20 00:04:00+00:00,23.91,24.47,3,2.34,284.96,9.37
...,...,...,...,...,...,...
2024-06-28 15:08:00+00:00,23.85,24.61,-5,3.19,-232.62,12.75
2024-06-28 15:09:00+00:00,23.84,24.61,-5,3.23,-235.78,12.92
2024-06-28 15:10:00+00:00,23.84,24.61,-5,3.23,-235.78,12.92


In [14]:
from_ts = ex.parse8601('2024-04-05 00:00:00')
ohlcv_list = []
ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=from_ts, limit=1000)
ohlcv_list.append(ohlcv)
while True:
    from_ts = ohlcv[-1][0]
    new_ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', since=from_ts, limit=1000,)
    ohlcv.extend(new_ohlcv)
    if len(new_ohlcv)!=1000:
    	break
 
 

NameError: name 'ex' is not defined

In [ ]:
dd

In [77]:
ccxt.deribit().fetch_ticker('BTC-FS-27SEP24_28JUN24')

{'symbol': 'BTC-FS-27SEP24_28JUN24',
 'timestamp': 1712785096183,
 'datetime': '2024-04-10T21:38:16.183Z',
 'high': 2724.5,
 'low': 2720.0,
 'bid': 2860.5,
 'bidVolume': 50000.0,
 'ask': 2872.5,
 'askVolume': 50000.0,
 'vwap': None,
 'open': None,
 'close': 2720.0,
 'last': 2720.0,
 'previousClose': None,
 'change': None,
 'percentage': None,
 'average': None,
 'baseVolume': None,
 'quoteVolume': 0.0,
 'info': {'best_bid_amount': '50000.0',
  'best_ask_amount': '50000.0',
  'implied_bid': '2855.0',
  'implied_ask': '2882.5',
  'combo_state': 'active',
  'best_bid_price': '2860.5',
  'best_ask_price': '2872.5',
  'max_price': '3268.0',
  'min_price': '2495.0',
  'last_price': '2720.0',
  'settlement_price': '2850.96',
  'instrument_name': 'BTC-FS-27SEP24_28JUN24',
  'mark_price': '2881.46',
  'index_price': '69698.26',
  'stats': {'volume_notional': '91010.0',
   'volume_usd': '91010.0',
   'volume': '0.0',
   'price_change': '0.0',
   'low': '2720.0',
   'high': '2724.5'},
  'state': '

In [19]:
binance.fetch_ticker('ETH/USDT')["last"]

3538.29

In [10]:
# show all rows
pd.set_option('display.max_rows', None)
df[df.strategy_name == "bnn_s_ACH"][df.metric == "adj_profit"]

/var/folders/4_/kgw4zz_50zn3r6p_mdtgrwgm0000gp/T/ipykernel_98276/3074275559.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.strategy_name == "bnn_s_ACH"][df.metric == "adj_profit"]


,strategy_name,name,ML:Params:P2,ML:Params:P1,metric,field_name,value
0,bnn_s_ACH,ML,0.1,0.6,adj_profit,value,-45.6397
1,bnn_s_ACH,ML,0.1,0.6,adj_profit,value,-45.6397
34,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
35,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
36,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
37,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
38,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
39,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
40,bnn_s_ACH,ML,0.1,0.7,adj_profit,value,29.8577
153,bnn_s_ACH,ML,0.2,0.2,adj_profit,value,17.8766


In [ ]:
ccxt.deribit().fetch_ticker("BTC-FS-27DEC24_12APR24")

In [50]:
ccxt.deribit().fetch_ticker("BTC-FS-27DEC24_12APR24")

{'symbol': 'BTC-FS-27DEC24_12APR24',
 'timestamp': 1712783021770,
 'datetime': '2024-04-10T21:03:41.770Z',
 'high': None,
 'low': None,
 'bid': 7434.5,
 'bidVolume': 300000.0,
 'ask': 7461.5,
 'askVolume': 50000.0,
 'vwap': None,
 'open': None,
 'close': 8037.5,
 'last': 8037.5,
 'previousClose': None,
 'change': None,
 'percentage': None,
 'average': None,
 'baseVolume': None,
 'quoteVolume': 0.0,
 'info': {'best_bid_amount': '300000.0',
  'best_ask_amount': '50000.0',
  'implied_bid': '7427.5',
  'implied_ask': '7467.5',
  'combo_state': 'active',
  'best_bid_price': '7434.5',
  'best_ask_price': '7461.5',
  'max_price': '8004.5',
  'min_price': '6938.5',
  'last_price': '8037.5',
  'settlement_price': '7367.68',
  'instrument_name': 'BTC-FS-27DEC24_12APR24',
  'mark_price': '7471.32',
  'index_price': '69770.65',
  'stats': {'volume_notional': '0.0',
   'volume_usd': '0.0',
   'volume': '0.0',
   'price_change': None,
   'low': None,
   'high': None},
  'state': 'open',
  'timestamp